## Embeddings

Similar words occur in similar contexts. This idea is used to map words to small vectors called *embeddings* which are going to be close to each other when words have similar meanings and far apart when they don't.

### Word2Vec